# *get_fence_array(patch_length, overall_length)* revisited
```python
def get_fence_array_from_padded_segment_with_stride(patch_len, start_seg, end_seg, fill_complete=False):
    
    return fence_array
```

## Compare padding vs stride in the context of registration intersection pairs

In [1]:
import time
nb_start_time = time.time()

import os
import tempfile
import sys

import numpy as np
import pandas as pd

import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import get_level_sizes_dict, get_sample_selection_mask


def check_patch_in_bounds(x, y, X_dim, Y_dim):
    """ Usage: TrueFalse = check_patch_in_bounds(x, y, X_dim, Y_dim)
                determine if the box is within the image
    Args:
        x:               a tuple, list or array (x_start, x_end)
        y:               a tuple, list or array (y_start, Y_end)
        X_dim:           a tuple, list or array (Image_X_start, Image_X_end)
        Y_dim:           a tuple, list or array (Image_Y_start, Image_Y_end)
    """
    if x[0] > x[1] or y[0] > y[1] or X_dim[0] > X_dim[1] or Y_dim[0] > Y_dim[1]:
        return False
    
    if x[0] >= X_dim[0] and y[0] >= Y_dim[0] and x[1] < X_dim[1] and y[1] < Y_dim[1]:
        return True
    
    else:
        return False

    
def im_pair_hori(im_0, im_1):
    """ Usage: new_im = im_pair_hori(im_0, im_1)
            combine a list of PIL images horizontaly
    """
    w0 = im_0.size[0]
    w = w0 + im_1.size[0] + 1
    h = max(im_0.size[1], im_1.size[1])

    new_im = tip.Image.new('RGB', (w, h) )
    box = (0, 0, w0, h)
    new_im.paste(im_0, box)
    
    box = (w0+1, 0, w, h)
    new_im.paste(im_1, box)

    return new_im

X_dim, Y_dim = (120, 1193), (59, 1200)
x = (120 + 25, 344)
y = (60+280, 284)
print(x, y)
x = list(x)
y = np.array(y)
check_patch_in_bounds(x, y, X_dim, Y_dim)

(145, 344) (340, 284)


False

In [2]:
test_data_dir = '../../DigiPath_MLTK_data/RegistrationDevData/'
os.listdir(test_data_dir)

offset_data_file = os.path.join(test_data_dir, 'wsi_pair_sample.csv')
if os.path.isfile(offset_data_file):
    offset_df = pd.read_csv(offset_data_file)

print('wsi_pair_sample.csv:\n',offset_df.iloc[0], '\n')

offset_x = offset_df['truth_offset_x'].iloc[0]
offset_y = offset_df['truth_offset_y'].iloc[0]
offset_x, offset_y = int(round(offset_x)), int(round(offset_y))

auto_x = offset_df['auto_offset_x'].iloc[0]
auto_y = offset_df['auto_offset_y'].iloc[0]
auto_x, auto_y = int(round(auto_x)), int(round(auto_y))

# print('\noffset_x, offset_y, auto_x, auto_y\n', offset_x, offset_y, auto_x, auto_y)

fixed_wsi = os.path.join(test_data_dir, '54742d6c5d704efa8f0814456453573a.tiff')
fixed_levels_dict = get_level_sizes_dict(fixed_wsi)
fixed_max_width = fixed_levels_dict['image_size'][0]
fixed_max_height = fixed_levels_dict['image_size'][1]
    
float_wsi = os.path.join(test_data_dir, 'e39a8d60a56844d695e9579bce8f0335.tiff')
float_levels_dict = get_level_sizes_dict(float_wsi)
float_max_width = float_levels_dict['image_size'][0]
float_max_height = float_levels_dict['image_size'][1]

    
run_parameters = {'wsi_filename': fixed_wsi, 
                  'wsi_floatname': float_wsi,
                  'thumbnail_divisor': 20, 
                  'patch_select_method': 'threshold_rgb2lab', 
                  'rgb2lab_threshold': 80, 
                  'image_level': 0, 
                  'patch_height': 224, 
                  'patch_width': 224, 
                  'threshold': 0, 
                  'offset_x': offset_x, 
                  'offset_y': offset_y}

for k, v in run_parameters.items():
    print('%25s: %s'%(k,v))
    
print('\n\nfixed_wsi image: 54742d6c5d704efa8f0814456453573a.tiff')
for k, v in fixed_levels_dict.items():
    print('%25s: %s'%(k,v))

print('\n\nfloat_wsi image: e39a8d60a56844d695e9579bce8f0335.tiff')
for k, v in float_levels_dict.items():
    print('%25s: %s'%(k,v))

wsi_pair_sample.csv:
 fixed_wsi         54742d6c5d704efa8f0814456453573a
float_wsi         e39a8d60a56844d695e9579bce8f0335
truth_offset_x                            -1617.86
truth_offset_y                             1672.74
auto_offset_x                             -1620.95
auto_offset_y                               1675.6
Name: 0, dtype: object 

             wsi_filename: ../../DigiPath_MLTK_data/RegistrationDevData/54742d6c5d704efa8f0814456453573a.tiff
            wsi_floatname: ../../DigiPath_MLTK_data/RegistrationDevData/e39a8d60a56844d695e9579bce8f0335.tiff
        thumbnail_divisor: 20
      patch_select_method: threshold_rgb2lab
        rgb2lab_threshold: 80
              image_level: 0
             patch_height: 224
              patch_width: 224
                threshold: 0
                 offset_x: -1618
                 offset_y: 1673


fixed_wsi image: 54742d6c5d704efa8f0814456453573a.tiff
               image_size: (145408, 83968)
              level_count: 10
       

In [3]:
def image_bounds_check(patch_box, image_bounds):
    """ Usage: TrueFalse = check_patch_in_bounds(patch_box, image_bounds)
    True if the patch is propperly ordered and within the image bounds
                
    Args:
        patch_box:      (x0, y0, x1, y1) tuple for upper left and bottom right locations
        image_bounds:   (X_min, Y_min, X_max, Y_max) upper left and bottom right image bounds
        
    Returns:
        check_passed:   True or False (if patch box is out of order or out of bounds)
        
    """
    x0, y0, x1, y1 = patch_box
    X_min, Y_min, X_max, Y_max = image_bounds
    if x0 > x1 or y0 > y1 or X_min > X_max or Y_min > Y_max:
        return False
    
    if x0 >= X_min and y0 >= Y_min and x1 < X_max and y1 < Y_max:
        return True
    
    else:
        return False

    
def get_strided_fence_array(patch_length, start_seg, end_seg):
    """ Usage: fence_array = get_fence_array(patch_length, overall_length)
        create a left-right set of pairs that descrete overall_length into patch_length segments

    Args:
        patch_length:   patch size - number of pixels high or wide
        patch_length:   overall number of pixels high or wide

    Returns:
        fence_array:    boundry values for each segment
        -----------:    [[left_0, right_0],
                         [left_1, right_1],
                         [left_2, right_2],... ]
    """
    #                               protect algorithmic assumption
    if end_seg - start_seg < 2 * patch_length:
        #                           Return One Patch
        return np.array([[start_seg, end_seg]])
    
    #                               determine the stride:
    overall_length = end_seg - start_seg
    n_fenced = np.ceil(overall_length / patch_length).astype(int)  # number of boxes
    stride = np.ceil(overall_length / n_fenced).astype(int)
    print('stride = ', stride)
    #                               initialize the return array:
    fence_array = np.zeros((n_fenced, 2)).astype(int)
    
    #                               fill in the beginning pair:
    fence_array[0,0] = start_seg
    fence_array[0,1] = start_seg + patch_length - 1
    
    #                               fill in the rest, including the last:
    for k in range(1, fence_array.shape[0]):    
        fence_array[k, 0] = fence_array[k-1, 0] + stride
        fence_array[k, 1] = fence_array[k, 0] + patch_length - 1
        
    #                               re-fill in the last one measured from the end:
    fence_array[k, 1] = end_seg - 1
    fence_array[k, 0] = end_seg - patch_length
    
    return fence_array

def get_patch_location_array(run_parameters):
    """ Usage: patch_location_array, thumb_location_array = get_patch_location_array(run_parameters)
    Note:  Faster if thumbnail_divisor equal to an (openslide object).level_downsamples value

    Args (run_parameters):  python dict.keys()
                                wsi_filename:           file name (with valid path)
                                patch_height:           patch size = (patch_width, patch_height)
                                patch_width:            patch size = (patch_width, patch_height)
                                thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                                patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
                                image_level:            openslide image pyramid level 0,1,2,...
                                image_start_x:          for image registration intersection
                                image_end_x:            for image registration intersection
                                image_start_y:          for image registration intersection
                                image_end_y:            for image registration intersection
                                
                            optional key:
                                threshold:              minimimum sum of thresholded image (default = 0)
                                
    Returns:
        patch_location_array:   full scale image location of the patches
        thumb_location_array:   thumb scale locations of the same patches
        
    """
    #                   initialize an empty return value
    patch_location_array = []
    thumb_location_array = []
    
    #                   local name the input variables
    wsi_filename = run_parameters['wsi_filename']
    
    thumbnail_divisor = run_parameters['thumbnail_divisor']
    patch_select_method = run_parameters['patch_select_method']
    
    patch_height = max(run_parameters['patch_height'], 1)
    patch_width = max(run_parameters['patch_width'], 1)
    
    image_level = run_parameters['image_level']
    
    start_col = max(run_parameters['image_start_x'], 0)
    end_col = run_parameters['image_end_x']
    
    start_row = max(run_parameters['image_start_y'], 0)
    end_row = run_parameters['image_end_y']
    
    #                   set defaults for newly added parameters
    if 'threshold' in run_parameters:
        threshold = run_parameters['threshold']
    else:
        threshold = 0

        
    #                     OpenSlide open                      Open Large Image file
    os_im_obj = openslide.OpenSlide(wsi_filename)
    obj_level_diminsions = os_im_obj.level_dimensions
    
    #                     Scale All to the full sized image, don't allow read outside image
    pixels_width = obj_level_diminsions[0][0] - 1
    pixels_height = obj_level_diminsions[0][1] - 1
    end_col = min(obj_level_diminsions[0][0] - 1, end_col)
    end_row = min(obj_level_diminsions[0][1] - 1, end_row)
    
    #                     Only the patch need scale for image_level
    patch_scale_multiplier = os_im_obj.level_downsamples[image_level]
    scale_patch_height = patch_height * patch_scale_multiplier
    scale_patch_width = patch_width * patch_scale_multiplier
    
    #                   get the start, stop locations list for the rows
    rows_fence_array = get_strided_fence_array(scale_patch_height, start_row, end_row)

    #                   get the start, stop locations list for the columns
    cols_fence_array = get_strided_fence_array(scale_patch_width, start_col, end_col)

    #                   get a thumbnail image for the patch select method - scaled to full size
    thumbnail_size = (pixels_width // thumbnail_divisor, pixels_height // thumbnail_divisor)
    
    #                   profile - LONG TIME HERE: best if thumbnail_divisor == a level_downsamples
    small_im = os_im_obj.get_thumbnail(thumbnail_size)
    
    #                     OpenSlide close                     Close Large Image file
    os_im_obj.close()
    
    
    #                   get the binary mask as a measure of image region content
    mask_im = get_sample_selection_mask(small_im, patch_select_method)

    #                   iterator for rows:  (top_row, bottom_row, full_scale_row_number)
    it_rows = zip(rows_fence_array[:, 0] // thumbnail_divisor,
                  rows_fence_array[:, 1] // thumbnail_divisor,
                  rows_fence_array[:, 0], 
                  rows_fence_array[:, 1])

    #                   variables for columns iterator
    lft_cols = cols_fence_array[:, 0] // thumbnail_divisor
    rgt_cols = cols_fence_array[:, 1] // thumbnail_divisor
    cols_start_array = cols_fence_array[:, 0]
    cols_end_array = cols_fence_array[:, 1]

    #                   iterate the rows and columns of patches
    for tmb_row_top, tmb_row_bot, row_sart, row_end in it_rows:
        #               iterator for cols:  (left_column, right_column, full_scale_column_number)
        it_cols = zip(lft_cols, rgt_cols, cols_start_array, cols_end_array)

        for tmb_col_lft, tmb_col_rgt, col_start, col_end in it_cols:

            #           if the sum of the mask elements is larger than the threshold...
            if (mask_im[tmb_row_top:tmb_row_bot, tmb_col_lft:tmb_col_rgt]).sum() > threshold:

                #       add the full scale patch box
                patch_location_array.append((col_start, row_sart, col_end, row_end))
                
                #       add the thumb scale patch box
                thumb_location_array.append((tmb_col_lft, tmb_row_top, tmb_col_rgt, tmb_row_bot))

    return patch_location_array, thumb_location_array

run_parameters['thumbnail_divisor'] = 16
run_parameters['image_level'] = 0

run_parameters['image_start_x'] = 0
run_parameters['image_end_x'] = 145408
run_parameters['image_start_y'] = 0
run_parameters['image_end_y'] = 83968

# T_er_F = image_bounds_check(box im_box)
im_box = (run_parameters['image_start_x'], 
          run_parameters['image_start_y'], 
          run_parameters['image_end_x'], 
          run_parameters['image_end_y'])

thmbnl_dvsr = run_parameters['thumbnail_divisor']

print('image_level ', run_parameters['image_level'])
print('thumbnail_divisor ',thmbnl_dvsr, '  thumb patch size ',run_parameters['patch_height'] // thmbnl_dvsr,'\n')
print('image_size ', fixed_levels_dict['image_size'])
print('thumb_size ', np.array(fixed_levels_dict['image_size']) // thmbnl_dvsr,'\n')


patch_location_array, thumb_location_array = get_patch_location_array(run_parameters)

print('%i patches'%(len(patch_location_array)))

n_2_show = min(5, len(patch_location_array))
print('\nfirst %i'%(n_2_show))
for k in range(n_2_show):
    if image_bounds_check(patch_location_array[k], im_box) == False:
        print('\tOut of Bounds full scale:', patch_location_array[k])
    else:
        print('\tfull scale:', patch_location_array[k])
        
    print('\tthumb scale:', thumb_location_array[k])
    
print('\nlast %i'%(n_2_show))
for k in range(len(patch_location_array)-1, len(patch_location_array) - n_2_show - 1, -1):
    if image_bounds_check(patch_location_array[k], im_box) == False:
        print('\tOut of Bounds full scale:', patch_location_array[k])
    else:
        print('\tfull scale:', patch_location_array[k])
        
    print('\tthumb scale:', thumb_location_array[k])

image_level  0
thumbnail_divisor  16   thumb patch size  14 

image_size  (145408, 83968)
thumb_size  [9088 5248] 

stride =  224
stride =  224
56852 patches

first 5
	full scale: (34496, 0, 34719, 223)
	thumb scale: (2156, 0, 2169, 13)
	full scale: (42336, 0, 42559, 223)
	thumb scale: (2646, 0, 2659, 13)
	full scale: (45024, 0, 45247, 223)
	thumb scale: (2814, 0, 2827, 13)
	full scale: (54208, 0, 54431, 223)
	thumb scale: (3388, 0, 3401, 13)
	full scale: (77504, 0, 77727, 223)
	thumb scale: (4844, 0, 4857, 13)

last 5
	full scale: (56896, 83743, 57119, 83966)
	thumb scale: (3556, 5233, 3569, 5247)
	full scale: (119392, 83552, 119615, 83775)
	thumb scale: (7462, 5222, 7475, 5235)
	full scale: (81984, 83552, 82207, 83775)
	thumb scale: (5124, 5222, 5137, 5235)
	full scale: (78400, 83552, 78623, 83775)
	thumb scale: (4900, 5222, 4913, 5235)
	full scale: (56896, 83552, 57119, 83775)
	thumb scale: (3556, 5222, 3569, 5235)


## Run history
```text
image_level  1
thumbnail_divisor  16 thumb patch size  14 

image_size  (145408, 83968)
thumb_size  [9088 5248] 

stride =  447
stride =  448
16822 patches

first 5
	full scale: (15680, 0, 16127, 447)
	thumb scale: (980, 0, 1007, 27)
	full scale: (27328, 0, 27775, 447)
	thumb scale: (1708, 0, 1735, 27)
	full scale: (34496, 0, 34943, 447)
	thumb scale: (2156, 0, 2183, 27)
	full scale: (41664, 0, 42111, 447)
	thumb scale: (2604, 0, 2631, 27)
	full scale: (42112, 0, 42559, 447)
	thumb scale: (2632, 0, 2659, 27)

last 5
	full scale: (119168, 83520, 119615, 83967)
	thumb scale: (7448, 5220, 7475, 5247)
	full scale: (86016, 83520, 86463, 83967)
	thumb scale: (5376, 5220, 5403, 5247)
	full scale: (83776, 83520, 84223, 83967)
	thumb scale: (5236, 5220, 5263, 5247)
	full scale: (81984, 83520, 82431, 83967)
	thumb scale: (5124, 5220, 5151, 5247)
	full scale: (78400, 83520, 78847, 83967)
	thumb scale: (4900, 5220, 4927, 5247)
    
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

image_level  0
thumbnail_divisor  16 thumb patch size  14 

image_size  (145408, 83968)
thumb_size  [9088 5248] 

stride =  224
stride =  224
57800 patches

first 5
	full scale: (34496, 0, 34719, 223)
	thumb scale: (2156, 0, 2169, 13)
	full scale: (42336, 0, 42559, 223)
	thumb scale: (2646, 0, 2659, 13)
	full scale: (45024, 0, 45247, 223)
	thumb scale: (2814, 0, 2827, 13)
	full scale: (54208, 0, 54431, 223)
	thumb scale: (3388, 0, 3401, 13)
	full scale: (77504, 0, 77727, 223)
	thumb scale: (4844, 0, 4857, 13)

last 5
	full scale: (119392, 83552, 119615, 83775)
	thumb scale: (7462, 5222, 7475, 5235)
	full scale: (86240, 83552, 86463, 83775)
	thumb scale: (5390, 5222, 5403, 5235)
	full scale: (84000, 83552, 84223, 83775)
	thumb scale: (5250, 5222, 5263, 5235)
	full scale: (81984, 83552, 82207, 83775)
	thumb scale: (5124, 5222, 5137, 5235)
	full scale: (94304, 83328, 94527, 83551)
	thumb scale: (5894, 5208, 5907, 5221)
     
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

image_level  1
thumbnail_divisor  8 thumb patch size  28 

image_size  (145408, 83968)
thumb_size  [18176 10496] 

stride =  447
stride =  448
19092 patches

first 5
	full scale: (9856, 0, 10303, 447)
	thumb scale: (1232, 0, 1287, 55)
	full scale: (15680, 0, 16127, 447)
	thumb scale: (1960, 0, 2015, 55)
	full scale: (16128, 0, 16575, 447)
	thumb scale: (2016, 0, 2071, 55)
	full scale: (16576, 0, 17023, 447)
	thumb scale: (2072, 0, 2127, 55)
	full scale: (17472, 0, 17919, 447)
	thumb scale: (2184, 0, 2239, 55)

last 5
	full scale: (119168, 83520, 119615, 83967)
	thumb scale: (14896, 10440, 14951, 10495)
	full scale: (89152, 83520, 89599, 83967)
	thumb scale: (11144, 10440, 11199, 10495)
	full scale: (86912, 83520, 87359, 83967)
	thumb scale: (10864, 10440, 10919, 10495)
	full scale: (86016, 83520, 86463, 83967)
	thumb scale: (10752, 10440, 10807, 10495)
	full scale: (83776, 83520, 84223, 83967)
	thumb scale: (10472, 10440, 10527, 10495)
```

In [4]:
for k, v in run_parameters.items():
    print(k,v)
    
print('\n%20s:'%('key_name'))
print('(fixed values)')
print('(float values)')

for k, v in fixed_levels_dict.items():
    print('\n%20s:'%(k))
    print(v)
    print(float_levels_dict[k])

wsi_filename ../../DigiPath_MLTK_data/RegistrationDevData/54742d6c5d704efa8f0814456453573a.tiff
wsi_floatname ../../DigiPath_MLTK_data/RegistrationDevData/e39a8d60a56844d695e9579bce8f0335.tiff
thumbnail_divisor 16
patch_select_method threshold_rgb2lab
rgb2lab_threshold 80
image_level 0
patch_height 224
patch_width 224
threshold 0
offset_x -1618
offset_y 1673
image_start_x 0
image_end_x 145408
image_start_y 0
image_end_y 83968

            key_name:
(fixed values)
(float values)

          image_size:
(145408, 83968)
(126976, 75776)

         level_count:
10
9

   level_downsamples:
(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0)
(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0)

    level_diminsions:
((145408, 83968), (72704, 41984), (36352, 20992), (18176, 10496), (9088, 5248), (4544, 2624), (2272, 1312), (1136, 656), (568, 328), (284, 164))
((126976, 75776), (63488, 37888), (31744, 18944), (15872, 9472), (7936, 4736), (3968, 2368), (1984, 1184), (992, 592), (496, 296))

In [5]:
patch_location_array[0]

(34496, 0, 34719, 223)